# Debug CLDS Model
This notebook debugs CLDS training with a single set of hyperparameters.

In [ ]:
import os
os.environ['XLA_PYTHON_CLIENT_ALLOCATOR'] = 'platform'
import sys
sys.path.insert(0, '..')

import jax
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import jax.random as jr
import numpy as np
import matplotlib.pyplot as plt

from dynamax.linear_gaussian_ssm.models import LinearGaussianConjugateSSM, ConditionallyLinearGaussianSSM
from dynamax.utils.utils import Tm_basis, rbf_basis

## Hyperparameters

In [ ]:
# Model hyperparameters
state_dim = 10
L = 7  # number of basis functions
kappa = 0.2  # lengthscale for RBF, smoothness for Fourier
sigma = 1.0  # amplitude
basis_type = 'rbf'  # 'rbf' or 'fourier'
has_dynamics_bias = True

# Data parameters
data_path = '/oak/stanford/groups/swl1/hdlee/crcns/U201202_01'
block_size = 8
standardize = True

# Training parameters
num_iters = 50  # reduced for debugging
seed = 10796
model_seed = 1234

## Load Data

In [ ]:
condition_to_n = {'top':0, 'top_left':1, 'left':2, 'bottom_left':3, 
                  'bottom':4, 'bottom_right':5, 'right':6, 'top_right':7}

# Load emissions and conditions
emissions_path = os.path.join(data_path, 'emissions_v4.npy')
conditions_path = os.path.join(data_path, 'conditions.npy')

emissions = jnp.load(emissions_path)
conditions = np.load(conditions_path, allow_pickle=True)
for c in condition_to_n.keys():
    conditions[np.where(conditions==c)] = condition_to_n[c]
conditions = jnp.array(conditions.astype(int))

print(f"Emissions shape: {emissions.shape}")
print(f"Conditions shape: {conditions.shape}")
print(f"Unique conditions: {jnp.unique(conditions)}")

In [ ]:
# Split data
num_conditions = len(np.unique(conditions))
num_blocks = len(emissions) // block_size
num_trials = num_blocks * block_size
emissions = emissions[:num_trials]
conditions = conditions[:num_trials]

# Create train/test split at block level
block_masks = jnp.ones(num_blocks, dtype=bool)
num_test_blocks = num_blocks // 6
key = jr.PRNGKey(seed)
test_idx = jr.choice(key, jnp.arange(8, num_blocks-8, dtype=int), shape=(num_test_blocks,), replace=False)
block_masks = block_masks.at[test_idx].set(False)

# Temporal indices (block-level)
block_id_nums = jnp.repeat(jnp.arange(num_blocks, dtype=float), block_size)
block_id_nums = block_id_nums / (num_blocks - 1)  # normalize to [0, 1]

trial_masks = jnp.repeat(block_masks, block_size)
train_conditions = conditions[trial_masks]
test_conditions = conditions[~trial_masks]

# Standardize
if standardize:
    train_obs_ = emissions[trial_masks]
    train_obs_mean = jnp.mean(train_obs_, axis=(0, 1), keepdims=True)
    train_obs_std = jnp.std(train_obs_, axis=(0, 1), keepdims=True)
    train_obs = (emissions - train_obs_mean) / train_obs_std
else:
    train_obs = emissions

_, sequence_length, emission_dim = train_obs.shape
test_obs = train_obs[~trial_masks]

print(f"Num blocks: {num_blocks}, Block size: {block_size}")
print(f"Num train trials: {trial_masks.sum()}, Num test trials: {(~trial_masks).sum()}")
print(f"Emission dim: {emission_dim}, Sequence length: {sequence_length}")
print(f"Unique block_id_nums (train): {len(jnp.unique(block_id_nums[trial_masks]))}")

## Initialize Model

In [ ]:
# Create basis functions
if basis_type == 'rbf':
    basis_funcs = rbf_basis(L, M_conditions=1, sigma=sigma, kappa=kappa)
else:
    period = 1.0 + 6.0 * kappa
    basis_funcs = Tm_basis(L, M_conditions=1, sigma=sigma, kappa=kappa, period=period)

print(f"Number of basis functions: {len(basis_funcs)}")

# Initialize model
model = ConditionallyLinearGaussianSSM(
    state_dim=state_dim,
    emission_dim=emission_dim,
    num_conditions=num_conditions,
    has_dynamics_bias=has_dynamics_bias,
    torus_basis_funcs=basis_funcs,
    num_trials=len(train_obs[trial_masks]),
)

key = jr.PRNGKey(model_seed)
params, props = model.initialize(key=key)

print(f"\nInitial emission weights shape: {params.emissions.weights.shape}")
print(f"  Expected: (L={len(basis_funcs)}, emission_dim={emission_dim}, state_dim={state_dim})")

## Diagnostic: Check Basis Functions

In [ ]:
# Evaluate basis functions at different time points
print("Basis function values at different time points:")
t_values = [0.0, 0.25, 0.5, 0.75, 1.0]
for t in t_values:
    phi = model.wpgs_C.evaluate_basis(t)
    print(f"  t={t:.2f}: {phi[:5]}... (sum={phi.sum():.3f})")

# Plot basis functions
t_range = jnp.linspace(0, 1, 100)
phi_values = jnp.array([model.wpgs_C.evaluate_basis(t) for t in t_range])

plt.figure(figsize=(10, 4))
for i in range(min(len(basis_funcs), 7)):
    plt.plot(t_range, phi_values[:, i], label=f'Basis {i}')
plt.xlabel('Time (normalized)')
plt.ylabel('Basis function value')
plt.title(f'{basis_type.upper()} Basis Functions (L={L}, kappa={kappa}, sigma={sigma})')
plt.legend()
plt.grid(True)
plt.show()

## Train Model

In [ ]:
# Train CLDS
best_params, train_lps = model.fit_em(
    params=params,
    props=props,
    emissions=train_obs[trial_masks],
    conditions=train_conditions,
    block_id_nums=block_id_nums[trial_masks],
    num_iters=num_iters,
    use_wandb=False,
)

print(f"\nFinal train log-likelihood: {train_lps[-1]:.2f}")

In [ ]:
# Plot training curve
plt.figure(figsize=(8, 4))
plt.plot(train_lps)
plt.xlabel('EM Iteration')
plt.ylabel('Log-likelihood')
plt.title('CLDS Training Curve')
plt.grid(True)
plt.show()

## Diagnostic: Check Learned Weights

In [ ]:
# Check emission weights
W_C = best_params.emissions.weights
print(f"Emission weights shape: {W_C.shape}")
print(f"Mean absolute weight per basis function:")
mean_abs_weights = jnp.abs(W_C).mean(axis=(1, 2))
for i, w in enumerate(mean_abs_weights):
    print(f"  Basis {i}: {w:.4f}")

In [ ]:
# Check emission matrix variation over time
print("\nEmission matrix norm at different times:")
C_matrices = []
for t in t_values:
    C_t = model.wpgs_C(best_params.emissions.weights, t)
    C_matrices.append(C_t)
    print(f"  t={t:.2f}: ||C|| = {jnp.linalg.norm(C_t):.4f}")

# Relative change from start to end
C_0 = C_matrices[0]
C_1 = C_matrices[-1]
rel_change = jnp.linalg.norm(C_1 - C_0) / jnp.linalg.norm(C_0)
print(f"\n||C(1) - C(0)|| / ||C(0)|| = {rel_change:.4f}")
print(f"  (This should be > 0.1 for meaningful time variation)")

## Compare with LDS

In [ ]:
# Train LDS for comparison
lds_model = LinearGaussianConjugateSSM(
    state_dim=state_dim,
    emission_dim=emission_dim,
    num_conditions=num_conditions,
    has_dynamics_bias=has_dynamics_bias,
)

key = jr.PRNGKey(model_seed)
lds_params, lds_props = lds_model.initialize(key=key)

best_lds_params, lds_train_lps = lds_model.fit_em(
    params=lds_params,
    props=lds_props,
    emissions=train_obs[trial_masks],
    conditions=train_conditions,
    num_iters=num_iters,
    use_wandb=False,
)

print(f"LDS final train log-likelihood: {lds_train_lps[-1]:.2f}")
print(f"CLDS final train log-likelihood: {train_lps[-1]:.2f}")
print(f"Difference (CLDS - LDS): {train_lps[-1] - lds_train_lps[-1]:.2f}")

In [ ]:
# Compare test log-likelihoods
test_block_ids = block_id_nums[~trial_masks]

clds_test_ll = model.batch_marginal_log_prob(
    best_params, test_obs, conditions=test_conditions, trial_ids=test_block_ids
)

lds_test_ll = lds_model.batch_marginal_log_prob(
    best_lds_params, test_obs, conditions=test_conditions
)

print(f"\nTest Log-Likelihoods:")
print(f"  LDS:  {lds_test_ll:.2f}")
print(f"  CLDS: {clds_test_ll:.2f}")
print(f"  Difference (CLDS - LDS): {clds_test_ll - lds_test_ll:.2f}")

In [ ]:
# Plot comparison
plt.figure(figsize=(10, 4))
plt.plot(train_lps, label='CLDS')
plt.plot(lds_train_lps, label='LDS')
plt.xlabel('EM Iteration')
plt.ylabel('Log-likelihood')
plt.title('Training Curves: CLDS vs LDS')
plt.legend()
plt.grid(True)
plt.show()

## Diagnostic: Regularization Analysis

In [ ]:
# Check if regularization is too strong
# The M-step uses: ZTZ + I (identity regularization with coefficient 1.0)

print("Regularization Analysis:")
print(f"  L * state_dim = {len(basis_funcs)} * {state_dim} = {len(basis_funcs) * state_dim}")
print(f"  Regularization coefficient: 1.0")
print(f"  If ZTZ diagonal << 1.0, regularization dominates!")
print(f"\nTo reduce regularization, edit models.py line 1715:")
print(f"  ZTZ + 1e-4 * jnp.eye(...)  # instead of ZTZ + jnp.eye(...)")